# Importing a CSV file into a PostgrSQL database

## Steps
- download the CSV file from the web using Python
- import a csv file into a pd dataframe 
- clean the table name and remove all the all extra symbols, spaces, capital letters
- clean the column headers and remove all the all extra symbols, spaces, capital letters
- write the create table SQL statement
- import the data into the db

In [1]:
# import libraries
import requests
import os
import numpy as np
import pandas as pd
import psycopg2
import shutil

In [2]:
# set working directory
os.chdir(r'C:\Users\manuk\OneDrive\Desktop\MasterSchool_final\Portfolio_projects\COVID_19')
os.getcwd()

'C:\\Users\\manuk\\OneDrive\\Desktop\\MasterSchool_final\\Portfolio_projects\\COVID_19'

In [3]:
# import the requests library, which allows us to make HTTP requests
import requests

# set the URL of the CSV file we want to download
url = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

# make a GET request to the URL, and store the response in a variable named 'response'
response = requests.get(url)

# open a file named 'owid-covid-data.csv' in binary write mode ('wb'), and store it in a variable named 'f'
with open("owid-covid-data.csv", "wb") as f:
    # write the content of the response to the file
    f.write(response.content)

In [4]:
# create a DataFrame from a CSV file named 'data.csv'
df = pd.read_csv("owid-covid-data.csv")
df.head()

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,population,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
0,AFG,Asia,Afghanistan,2020-01-03,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
1,AFG,Asia,Afghanistan,2020-01-04,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
2,AFG,Asia,Afghanistan,2020-01-05,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
3,AFG,Asia,Afghanistan,2020-01-06,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN
4,AFG,Asia,Afghanistan,2020-01-07,NaN,0.0,NaN,NaN,0.0,NaN,...,NaN,37.746,0.5,64.83,0.511,41128772.0,NaN,NaN,NaN,NaN


In [5]:
# loop over the columns of the DataFrame 'df'
for i in df.columns:
    # print the name of each column
    print(i)

iso_code
continent
location
date
total_cases
new_cases
new_cases_smoothed
total_deaths
new_deaths
new_deaths_smoothed
total_cases_per_million
new_cases_per_million
new_cases_smoothed_per_million
total_deaths_per_million
new_deaths_per_million
new_deaths_smoothed_per_million
reproduction_rate
icu_patients
icu_patients_per_million
hosp_patients
hosp_patients_per_million
weekly_icu_admissions
weekly_icu_admissions_per_million
weekly_hosp_admissions
weekly_hosp_admissions_per_million
total_tests
new_tests
total_tests_per_thousand
new_tests_per_thousand
new_tests_smoothed
new_tests_smoothed_per_thousand
positive_rate
tests_per_case
tests_units
total_vaccinations
people_vaccinated
people_fully_vaccinated
total_boosters
new_vaccinations
new_vaccinations_smoothed
total_vaccinations_per_hundred
people_vaccinated_per_hundred
people_fully_vaccinated_per_hundred
total_boosters_per_hundred
new_vaccinations_smoothed_per_million
new_people_vaccinated_smoothed
new_people_vaccinated_smoothed_per_hundre

In [6]:
# create the covid_deaths dataframe with columns until weekly_hosp_admissions_per_million
covid_deaths = df.loc[:, :'weekly_hosp_admissions_per_million']
covid_deaths.insert(3, 'population', df['population'])  # add the 'population' column after 'date'

# create the covid_vaccinations dataframe with columns from total_tests to the end
covid_vaccinations = df.loc[:, 'total_tests':]
covid_vaccinations = covid_vaccinations.drop('population', axis=1)  # remove the 'population' column from the second dataframe

# add the first 4 columns from df to covid_vaccinations
covid_vaccinations = pd.concat([df.iloc[:, :4], covid_vaccinations], axis=1)

# save the dataframes to CSV files
covid_deaths.to_csv('covid_deaths.csv', index=False)
covid_vaccinations.to_csv('covid_vaccinations.csv', index=False)

## Find CSV files in directory

In [7]:
# find CSV files in my currecnt working directory
# isolate only CSV files

csv_files=[]
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [8]:
# make a new directory

dataset_dir = 'datasets'

# create the bash command to make a new directory
#  mkdir dataset_dir

try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [9]:
# construct the full path of the file in the main folder
data_path = os.getcwd() + '\\' + dataset_dir + '\\'

In [10]:
# loop through all files in the main folder
for file_name in os.listdir(os.getcwd()):
    # check if the file is a CSV file
    if file_name.endswith('.csv'):
        # construct the full path of the file in the main folder
        file_path = os.path.join(os.getcwd(), file_name)

        # construct the full path of the file in the datasets subfolder
        new_file_path = os.path.join(data_path, file_name)

        # use the shutil module's move() function to move the file
        shutil.copy(file_path, new_file_path)

In [11]:
data_path = os.getcwd() + '/' + dataset_dir + '/'

df = {}
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
    print(file)

covid_deaths.csv
covid_vaccinations.csv
owid-covid-data.csv


## Clean table names and column names

In [12]:
for k in csv_files:
    dataframe = df[k]
    
    clean_tbl_name = k.lower().replace(" ", "_").replace("-","_")\
                    .replace(r"/","_").replace("\\","_")\
                    .replace("$","").replace("%","")
    
    #remove .csv extension from clean_tbl_name
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    
    dataframe.columns = [x.lower().replace(" ", "_").replace("-","_")\
                    .replace(r"/","_").replace("\\","_")\
                    .replace("$","").replace("%","") for x in dataframe.columns]   
    
    #replacment dictionary that maps pandas dtypes to sql dtypes
    replacements = {
    'object':'varchar',
    'float64':'float',
    'int64':'int',
    'datetime64':'timestamp', 
    'timedelta64[ns]':'varchar'
}    
    
    # table schema
    
    # create a list of formatted column names and data types
    columns = ["{} {}".format(n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements))]

    # join the list into a comma-separated string
    col_str = ", ".join(columns)
    
    # open a database connection
    host='localhost'
    dbname='PortfolioProjects'
    user='postgres'
    password='elmanlus84'


    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, user, password)

    conn = psycopg2.connect(conn_string)
    cursor = conn.cursor()
    print('opened database successfully')
    
    # drop tables with same name
    cursor.execute('drop table if exists %s;' % (tbl_name))    
    # creat table
    cursor.execute('create table %s (%s)' % (tbl_name,col_str))
    print('{0} was created successfully'.format(tbl_name))
    
    # insert values to table

    # save the df to csv
    dataframe.to_csv(k, header=dataframe.columns, index=False, encoding='utf-8')

    # open the csv file, save it as an object
    my_file = open(k)
    print('file opened in memory')
    
    
    # upload to db
    SQL_statement = """

    COPY %s FROM STDIN WITH
    CSV
    HEADER
    DELIMITER AS ','

    """

    cursor.copy_expert(sql=SQL_statement % tbl_name, file=my_file)
    print('file copied to db')
    
    conn.commit()
    conn.close()
    print('table {0} imported to db completed'.format(tbl_name))
    
#for loop end message
print('all tables have been successfully imported into the db')

opened database successfully
covid_deaths was created successfully
file opened in memory
file copied to db
table covid_deaths imported to db completed
opened database successfully
covid_vaccinations was created successfully
file opened in memory
file copied to db
table covid_vaccinations imported to db completed
opened database successfully
owid_covid_data was created successfully
file opened in memory
file copied to db
table owid_covid_data imported to db completed
all tables have been successfully imported into the db
